# Mojo notes

These notes will be in md format until the mojo SDK is released in September.

This document is currently written for readers who already have experience with a programming language, so it will not
start from first principles.  There will be frequent references to rust, but it is not required to know rust.

There will also be frequent comparisons to python, since mojo aims to be a superset of python (it is not close to this
yet however).  Therefore, ideally, you should know python as well.  However, this is also not strictly necessary, and
overtime, I will write the document to teach both python and mojo basics as mojo gains more compatibility with python.

## Some limitations to note in mojo

While mojo aims to be a superset of python, it is not yet there.  Some current limitations are the following:

- No top-level variables (ie, variables that don't live inside a function or struct)
- No python `class` type (but it does have `struct` which is the non-dynamic version of a class)
- The SDK to run locally is only available on x86_64 for linux (ubuntu)
- `def` is not fully equivalent to python's def yet (eg, mojo's scope is different)
- See [proposal on dynamism](https://github.com/modularml/mojo/blob/main/proposals/mojo-and-dynamism.md) for more
  details on mojo's lack of dynamism
- Keyword arguments in mojo do not work (so you can't pass them to python either)
    - This also means dict unpacking does not work (eg **kwargs, or *args for that matter)
- For a more thorough list, see the [Roadmap and Rough Edges] documentation

## Functions in Mojo

The bread and butter of almost all languages (java being a lonely outlier) are functions, and mojo is no exception.  In mojo,
there are two ways to define a function:

- like python with `def`
- or with `fn`

> **Parameter vs Argument**
> 
> Many languages treat parameters and arguments as synonyms, but mojo has a more precise definition.  A parameter is what is in
> the declaration of the function and is therefore compile time.  An argument is the runtime type/value that gets passed into
> the function when it is called, and is therefore known at runtime.  This dovetails nicely with mojo's Parameterized Expressions
> which are like generics and functions that can run at compile time.

If using a `def` definition, type annotations for are not required (though still recommended) for a function's parameters or in the
body of the function.  If using a `fn` definition, then the type annotations are required both for parameters, and for any locally
declared variables in the body.

Mojo uses a slightly modified version of the PEP-695 syntax (forthcoming in python 3.12 due out in early October 2023).  Unlike
python type annotations where the return type can be inferred, in mojo, you must still specify the return type if using `fn` 
definition.

> **Async**: Mojo also has support for `async fn` and `async def` as well, along with the `await` keyword.

Here is an example of a simple mojo function

In [ ]:
fn my_relu(x: Int, y: Int = 10) -> Int:
    let result: Int
    if x >= y:
        result = x * 2
    else:
        result = x
    return result

### Declaring (Im)Mutability: let and var

In mojo fn's or struct fields, you must declare local variables either with `let` or `var`.

- `let` is for immutable variables
- `var` is for mutable variables

In rust, `let` and `let mut` would be the equivalent identifiers.

> **Mojo Update**
> According to the mojo proposals for keyword renaming, the `let` may be renamed, and `var` might serve double duty

## Argument passing

Some programmers may not even understand what _argument passing_ means.  Some languages have only one way to do it, like
python.  Or it may have something awkward like java (which differentiates between Objects and primitives) and do
Boxing and Unboxing when the arguments are primitives, thereby causing a performance hit.  And since (Un)Boxing is 
implicit, it is not obvious that there are different ways to pass args to java methods.

So if you are not familiar with the terms, there are generally two different kinds of argument passing:

- by value: (sometimes called by-copy) where the value of a variable is used
- by reference: where the value is indirectly retrieved through reference or pointer dereferencing

This is sometimes confusing to people, because pointers or references can be hard to conceptualize.  If it helps, think
about _value equality_ versus _identity equality_ which is what Java does.  By default, in java, the `==` operator
compares _identity equality_, meaning "does this variable point to the same memory as this other variable?".  But 95% of
the time what we care about is _value equality_, or "does this variable have the same contents of this other variable?".

Also, a brief discussion of `ownership` needs to be mentioned.  Similarly to rust, mojo has a concept of ownership which
basically just means "who is responsible for cleaning up this data?".  In languages without Garbage Collectors, this is
very important (and is what C and C++ lack).

> **Mojo Update**
> 
> There is some discussion on whether to rename some keywords. Notably `borrowed` may become `ref` and `inout` might
> become `refmut`.  This is due to some changes they may need to make for `lifetimes`.  Also, they will probably change
> they keywords to _modify the type_ rather than modify the parameter _name_.  For example:
> ```python
> # Current way
> fm modify_employee(inout employee: Employee): ...
> # Proposed way
> fn modify_employee(employee: refmut Employee): ...
> ```

In mojo, by default, all parameters are immutable and passed by reference (an implicit keyword `borrowed` is added as a 
prefix to the parameter name so that `foo(borrowed age: Int)` and `foo(age: Int)` are equivalent). However, there are 
several ways to pass arguments to functions in mojo. An argument to a function can be:

- **moved**: (aka _transferred_) where the ownership and _value_ of the argument is transferred to the function
- **by reference (immutably)**: where a shared reference to the argument object is passed in 
- **by reference (mutably)**: where an exclusive mutable reference to the argument object is passed in

   
- **by reference (mutably)**: where an exclusive mutable reference to the argument object is passed in


### Pass by value: Move

You can call a function with pass-by-value through move semantics.  Rather than copying the variable's data into the
function, the data is transferred (ie, _moved_) from the variable into the called function.  This means that after the
function is done, the original variable can't be used.

- The type of the parameter must have a `__moveinit__` method defined
- In the fn declaration, the parameter name will be prefixed with `owned` keyword
- On the caller side, the argument is postfixed with the `^` symbol (eg `foo(age^)`)
- Once moved, the original variable is no longer accessible (technically, it could be zeroed out but cant be reached)
- _moves_ also happen in assignment, when the RHS variable is post-fixed with the `^` sigil (`let obj2 = obj1^`)

> Comparison: Rust Move
>
> Note that the default behavior is opposite to rust.  In rust, the default is to pass by value and to _move_ it (the
> data is transferred and then once out of scope is gone).  Mojo takes the stand that it is more common to want to pass
> by (immutable) reference and makes this the default.  This is also more similar to how python works, although most
> things in python are a mutable pass by reference (eg, lists, dicts, and most classes). You therefore do not need to
> mark the argument with a sigil (like `&foo`) that rust requires when you _do_ want to pass by immutable reference
> (they did however consider this, and may reintroduce it depending on how their lifetime system works).

In [ ]:
# This is an example of a move
# For the following code to work, MyStruct must implement __moveinit__

struct MyStruct:
    var value: Pointer[]

    def __moveinit__(inout self, rhs: Self):


fn foo(owned data: MyStruct):
    # do something with data
    ...

let obj = MyStruct()
foo(obj^)  # think of the ^ like it's being given up
print(obj) # will fail, because obj was moved

# Or in assignment
let obj2 = MyStruct()
# after this, obj2 is no longer accessible.  
let obj3 = obj2^  

> **Mojo Update**
> >
> There is some discussion on whether to rename some keywords. Notably `borrowed` may become `ref` and `inout` might
> become `refmut`.  This is due to some changes they may need to make for `lifetimes`.  Also, they will probably change
> they keywords to _modify the type_ rather than modify the parameter _name_.  For example:
> ```python
> # Current way
> fm modify_employee(inout employee: Employee): ...
> # Proposed way
> fn modify_employee(employee: refmut Employee): ...
> ```

### Pass by immutable reference

The default in mojo is to pass arguments by immutable reference. Some things to note about passing by immutable 
reference (aka borrowed):

- This is the default, and an implicit `borrowed` keyword is prefixed before the parameter name
- No mutation of the argument can occur
- the compiler will keep track that the argument object has been borrowed immutably
- Since the reference is obtained immutably, there can be many other immutable borrows
    - ie, the argument could be passed immutably to another fn


The following shows a simple example

In [ ]:
#  Example of pass by immutable reference

def double_by_ref(data: MyStruct):
    # data.value = 10 # This will fail because we are trying to mutate data
    return data.value * 2

obj = MyStruct()
print(foo_by_ref(obj))
print(obj)  # This still works, because we passed by reference so `obj` is still alive

### Pass by mutable reference

Sometimes, you want to pass by mutable reference.  Any changes that happen inside the function body will be seen outside
of the function too.  This is the default in python (when the object is mutable like a list or dict)

- The fn declaration parameter name is prefixed with `inout`  (eg `foo(inout age: Int)`)
- Since the argument value passed in can be directly mutated, it is **not** a copy of the data
- Since it is a mutable reference, only one such mutable (aka exclusive) reference can exist in the lifetime of the use
  of the function

In [ ]:
# Example of pass by mutable reference using the `inout` modifier
def foo_by_mut_ref(inout data: MyStruct):
    data.value = 10

obj = MyStruct()
foo_by_mut_ref(obj)  # obj still exists and was mutated

## Copy vs Move vs Reference

Unlike rust, mojo allows you to make types moveable or not (in rust, the affine type system requires all types to
transfer the data and then effectively delete the old value, placing a lot of stress on memcpy performance).  In rust,
one usually adds the `Copy` and/or `Clone` auto derived traits to a type to make it copy-able (shallow or value-wise) or
cloneable (deep...for copying ref data).

In mojo, copying is done manually by implementing a `__copyinit__` dunder method.  It is optional, but without an
implementation, you can not put the type on the right hand side of an assignment.

> **Note**: I will use the acronym RHS for right hand side, and LHS for left hand side 

After an assignment where the RHS type implements `__copyinit__`, the variable on the RHS is still available.  This is
like a rust type that implements the `Copy` trait.  If a rust type does not implement the `Copy` trait, then the 
variable RHS would have been _moved_ into the variable on the LHS.

I try to analogize that `__copyinit__` and `__moveinit__` are like the unix `cp` and `mv` commands respectively.
With the `cp` command, the original source file will still exist after being called, whereas in the `mv` command,
the source file will no longer exist.

Here's an example of a data type that has no copy or move constructors.

In [ ]:
# A type without a copy or move constructor
struct Foo:
    let age: Int

    def __init__(inout self, age: Int):
        self.age = age

obj = Foo(10)
let obj2 = obj  # will fail here, because this would be a copy and there is no __copyinit__ implememted

### Why copies, moves and refs?

One may (and should) be asking why all the different kinds of passing arguments into functions?

> **Food for thought**
>
> In rust, you don't have a choice whether a move will happen or not.  What about mojo? So this begs a couple of
> questions:
>
> - What happens if you have neither a `__copyinit__` or `__moveinit__` defined?
> - Why wouldn't you want a `__moveinit__` (or `__copyinit__`) defined?
> - What exactly is moved?

To answer the questions above, we need to consider what a variable _really_ is.  We will cover this in a later section.

In a move, the value stored at the memory location of the RHS is copied over to the LHS, and then the memory for the RHS
is deleted. In rust's case, the drop doesn't actually happen until the end of the scope, whereas in mojo it's as soon as 
the variable is no longer used anymore...even within the same scope.  This has a couple of advantages for mojo compared 
to rust by eliminating the need for [dynamic drop flags](https://doc.rust-lang.org/nomicon/drop-flags.html).

But what about references?  Currently, mojo doesn't have explicit references.  They do have `ref` and `mut ref` as
reserved keywords, but they are still working on fleshing out their lifetime system.  That being said, whenever you pass
a variable into a function, by default, it's being passed in as an immutable reference (the data is neither being copied
nor moved into the function).  Unless a parameter is marked with `inout`, so that it is a mutable reference, or with the
`owned` prefix, so that the variable itself is moved, the argument is passed in by immutable reference.

> **But Why?**
>
> The reason mojo choose this as the default, is that they believe this is the more common scenario, and it (somewhat)
> dovetails with how pythonistas program.  Since everything is an object (lives on the heap) in python, everything in 
> python is passed by reference.  Whether it is a mutable or immutable reference depends on the type in python.

In rust, it defaults to move semantics, and therefore pass by value.